Calvin Passmore

ECE 5660

# Homework 3

## Problem 1

Write Python code to produce phase trajectory plots (see p. 236). Reproduce
the four trajectory plots in figure 5.3.14.

---

See attached code

The four plots are below

![](./images/Phase_Trajectory_4QAM_ALPHA1.png)

![](./images/Phase_Trajectory_4QAM_ALPHA0.5.png)

![](./images/Phase_Trajectory_16QAM_ALPHA1.png)

![](./images/Phase_Trajectory_16QAM_ALPHA0.5.png)

---
---

## Problem 2

Write a general QAM transmit-receive system simulator in Python.
For now, neglect additive noise in the channel, and assuming that timing synchronization is known.
You are essentially building the diagram in Fig. 5.3.16 (a) for the transmitter, and Fig. 5.3.16(b) for the receiver.
(But you won’t have a BPF or ADC in your receiver, and you won’t need a DDS, since you can just compute sin and cos directly in your programming language.)
The parameters you should be able to change easily are:
- Ts the symbol period
- N the number of samples per symbol (use the same N for transmitter and receiver).
- p(t), p(−t) the pulse shape and the matched filter. For the SRRC pulse, the excess bandwidth α should be a settable parameter.
- &Omega;<sub>c</sub> the carrier frequency
- The number of points in the signal constellation
- LUT, the LUT used to determined the signal constellation

---

See the attached code, and note the parameters on the function <i>transmit_and_receive</i>

---
---

## Problem 3

Using your software, simulate QPSK and the constellation in Fig. 5.3.8(b).
Use a SRRC pulse with α = 1.

(a) (5 pts) Plot I/Q phase trajectory plots the receiver, analogous to Fig. 5.3.14.

(b) (5 pts) Plot the eye diagrams for both the inphase and quadrature signals.

(c) (5 pts) Plot a scatter plot of the sampled matched filter outputs.
This is the x-y plot of the points obtained from the matched filter outputs.
In the absence of noise, these should all fall on the constellation points.

(d) (10 pts) Plot the power spectra of the I(t) signal and the modulated signal s(t) = I(t)√2 cos(Ωcn) −Q(t)√2 sin(Ωcn) and the matched filter output.

You can plot the power spectrum as follows:
- Compute the autocorrelation function.
- Compute the FFT of the autocorrelation function.
- Plot the magnitude square of the FFT. It is helpful to do this in dB. You may want to “fftshift” the axis to better understand the frequency response.
- As a hint, here is a 1-liner that does the plot in Matlab. (The x axis should be adjusted so that it shows frequency scale.)

<i>plot(10*log10(fftshift(abs(fft(xcorr(s))).ˆ2)))</i>

You can convert this to Python. See numpy.correlate, numpy.fft.fft,
numpy.fft.fftshift.
For example,

<i>plt.plot(10*np.log10(np.abs(np.fft.fftshift( np.fft.fft(np.correlate(y,y,’full’))))**2))</i>

Make sure you study these examples enough to understand what is going on!

---

(a)

![](./images/Phase_TrajectoryQPSK.png)

(b)

![](./images/Eye_Diagram_Inphase_QPSK.png)

![](./images/Eye_Diagram_Quadurature_QPSK.png)

(c)

![](./images/Scatter_PlotQPSK.png)

(d)

![](./images/Power_Spectra_Ir_QPSK.png)

![](./images/Power_Spectra_s(t)_QPSK.png)

![](./images/Power_Spectra_x(t)_QPSK.png)

---
---

## Problem 4

Test your QPSK simulation as follows: Generate 1000 pairs of bits.
(To generate pairs of bits, you may simply want to generate integers representing bit pairs, taking on the values 0, 1, 2, 3, since this can be thought of as the output of the S/P block.)
Modulate these bits in your transmitter system, producing the signal s(n).
Pass s(n) into your receiver.
Take the output of your slicer block (which slices matched filter outputs into symbols), convert the symbols back into bits (or pairs of bits, in your case).
Then compare the bit pairs at the output of your receiver with the bit pairs at the input of your transmitter.
Ensure that all the bit pairs are correct.
(You will need to get the timing right at the the receiver to align the output bits with the input bits.)

---

See the attached code for the generation of this line, the output of comparing bits in and bits out is:

    Do the transmitted bits match the recevied bits? True


## The Code

### HW3.py

```
import numpy as np
from numpy.random import rand
from pulses import *
from plots import *
from tx import *
from rx import *

def transmit_and_receive(bits, Nsym, LUT, bits_per_sym, pulse, Lp, Ts, omega, keyword=''):

    parallel = serial_to_parallel(bits, bits_per_sym)
    # plot_and_show(parallel, title="Parallel", plot='stem')

    I,Q = parallel_to_I_Q(parallel, LUT)
    # plot_and_show(I, title="I(t)", plot='stem')
    # plot_and_show(Q, title="Q(t)", plot='stem')
    # plot_and_show(Q, I, title="I vs Q")

    I_upsampled = upsample(I, N)
    Q_upsampled = upsample(Q, N)
    # plot_and_show(I_upsampled, title="I(t) upsampled", plot='stem')
    # plot_and_show(Q_upsampled, title="Q(t) upsampled", plot='stem')

    I_shaped = np.convolve(I_upsampled, pulse)
    Q_shaped = np.convolve(Q_upsampled, pulse)
    # plot_and_show(I_shaped, title="I shaped")
    # plot_and_show(Q_shaped, title="Q_shaped")

    ######################################
    #TODO: Send and receive with cos and sin
    ######################################

    Ir = I_shaped
    Qr = Q_shaped

    I_sig = [i * sqrt(2) *  cos(omega) for i in I_shaped]
    Q_sig = [q * sqrt(2) * -sin(omega) for q in Q_shaped]

    st = [I_sig[i] + Q_sig[i] for i in range(len(I_sig))]
    plot_and_show(st, title="Signal")

    # rt = st # Transmit over the channel, noiseless

    # Ir = [i * sqrt(2) *  cos(omega) for i in rt]
    # Qr = [q * sqrt(2) * -sin(omega) for q in rt]

    xt = matched_filter(Ir, pulse)
    yt = matched_filter(Qr, pulse)

    # plot_and_show(xt, title="x(t)")
    # plot_and_show(yt, title="y(t)")

    xt_sampled = sample_signal(xt, sample_time=N, Lp=Lp)
    yt_sampled = sample_signal(yt, sample_time=N, Lp=Lp)

    # plot_and_show(xt_sampled, title="Sampled x(t)", plot='stem')
    # plot_and_show(yt_sampled, title="Sampled y(t)", plot='stem')

    xt_normalized = normalize_amplitude(xt_sampled, max(max(LUT)))
    yt_normalized = normalize_amplitude(yt_sampled, max(max(LUT)))

    # plot_and_show(xt_normalized, title="Noramlized x(t)", plot='stem')
    # plot_and_show(yt_normalized, title="Normazlied y(t)", plot='stem')

    ###################
    # Phase Trajectory
    ####################
    plot_and_show(y=yt_normalized, x=xt_normalized, title="Phase_Trajectory" + keyword, xlabel="x(t)", ylabel="y(t)", axis=True)

    ###################
    # Scatter Plot
    ####################
    plot_and_show(y=yt_normalized, x=xt_normalized, title="Scatter_Plot" + keyword, plot='scatter', xlabel="x(t)", ylabel='y(t)', axis=True)

    ###################
    # Eye diagram
    ###################
    eye_diagram(xt, Lp, N, name=f"Inphase_{keyword}")
    eye_diagram(yt, Lp, N, name=f"Quadurature_{keyword}")

    ##################
    # Power Spectra
    ##################
    power_spectra(Ir, name="Ir_"+keyword)
    power_spectra(st, name="s(t)_"+keyword)
    power_spectra(xt, name="x(t)_"+keyword)

    bits_received = slice(xt_normalized, yt_normalized, LUT)
    return bits_received

# Constants
ALPHA_1=1
ALPHA_HALF=0.5
LUT_2PAM = [-1,1]
LUT_4PAM = [-2,-1,1,2]
LUT_4QAM = [[-1,-1], [-1,1], [1,-1], [1,1]]
LUT_16QAM = [[-3,-3], [-3,-1], [-3,3], [-3,1], [-1,-3], [-1,-1], [-1,3], [-1,1], [1,-3], [1,-1], [1,3], [1,1], [3,-3], [3,-1], [3,3], [3,1]]

# Contant variables
Nsym = 500
N = 1000
Lp = 6000
Ts = 1
omega = 100

for LUT, bits_per_sym in [[LUT_16QAM, 4], [LUT_4QAM, 2]]:
    for alpha in [ALPHA_1, ALPHA_HALF]:
        pulse = srrc1(alpha, N, Lp, Ts)
        if len(LUT) == 1:
            PQ = 'PAM'
        else:
            PQ = 'QAM'
        keyword = f'_{len(LUT)}{PQ}_ALPHA{alpha}'

        bits = (rand(Nsym*(bits_per_sym))> 0.5).astype(int) # generate random bits {0,1}

        bits_received = transmit_and_receive(bits=bits, Nsym=Nsym, LUT=LUT, bits_per_sym=bits_per_sym, pulse=pulse, Lp=Lp, Ts=Ts, omega=omega, keyword=keyword)

        print(f"Do the transmitted bits match the recevied bits? {np.array_equiv(bits, bits_received)}")

# For QPSK
bits = (rand(Nsym*(bits_per_sym))> 0.5).astype(int) # generate random bits {0,1}
N = 100
Lp = 600
Ts=1
omega=100
srrcout = srrc1(alpha=1, N=N, Lp=Lp, Ts=Ts)
bits_received = transmit_and_receive(bits=bits, Nsym=1000, LUT=LUT_4QAM, bits_per_sym=2, pulse=srrcout, Lp=Lp, Ts=Ts, omega=omega, keyword='QPSK')
print(f"Do the transmitted bits match the recevied bits? {np.array_equiv(bits, bits_received)}")

```

### plots.py

```
import matplotlib.pyplot as plt
import numpy as np
import time
import sys

def eye_diagram(sig:np.array, Lp, N, name=''):
    offset = (2*Lp - np.floor(N/2)).astype(int)
    Nsymtoss = 2*np.ceil(Lp/N) # throw away symbols at the end
    nc = (np.floor((len(sig) - offset - Nsymtoss*N)/N)).astype(int) # number of points of signal to plot
    xreshape = sig[offset:offset + nc*N].reshape(nc,N)
    plt.figure(5); plt.clf()
    plt.plot(np.arange(-np.floor(N/2), np.floor(N/2)), xreshape.T, color='b', linewidth=0.25)
    plt.title(f"Eye Diagram {name}")
    plt.savefig(f"images/Eye_Diagram_{name}.png")
    # plt.show()
    plt.close()

def power_spectra(sig, name=''):
    plt.figure()
    plt.plot(10*np.log10(np.abs(np.fft.fftshift(np.fft.fft(np.correlate(sig,sig,'full'))))**2))
    plt.title(f"Power Spectra {name}")
    plt.savefig(f"images/Power_Spectra_{name}.png")
    plt.close()

def plot_and_show(y, x:list=[], title:str=None, xlabel=None, ylabel=None, grid=None, plot=None, axis=False):
    plt.figure()

    if not plot:
        plot_type = plt.plot
    elif plot == 'stem':
        plot_type = plt.stem
    elif plot == 'scatter':
        plot_type = plt.scatter
    else:
        print(f"Unkown plot type {plot}")
        sys.exit(1)

    if x == []:
        plot_type(y)
    else:
        plot_type(y,x)

    if title:
        plt.title(title)

    if xlabel:
        plt.xlabel(xlabel)

    if ylabel:
        plt.ylabel(ylabel)

    if grid == 'log':
        plt.grid(which='both')
        plt.scale('log')
    elif grid == True:
        plt.grid()

    if axis:
        plt.axhline(0, color='black', linewidth=.5)
        plt.axvline(0, color='black', linewidth=.5)

    # plt.show()
    if not title:
        plt.savefig(f"images/{time.time()}.png")
    else:
        plt.savefig(f"images/{title}.png")

    plt.close()
```

### pulses.py

```
from math import sqrt, pi, sin, cos, floor

def srrc1(alpha, N, Lp, Ts):
    """
    Return a vector of the srrc function
    alpha = excess bandwidth
    N = samples per symbol
    Lp = SRRC truncation length #Currently only supports even numbers
    Ts = sample time
    """

    times = []
    number_of_samples = int(floor(Lp/2)) # and then reflect it on the axis?
    for idx in range(number_of_samples):
        t = idx * Ts / N
        times.append(-t)
        times.append(t)
    times.remove(0) # Remove the second zero
    times.sort()

    answer = []
    for t in times:
        answer.append(p_of_nT(Ts, alpha, t))

    while None in answer:
        index = answer.index(None)
        value = (answer[index-1] + answer[index+1])/2
        answer[index] = value

    return answer


def p_of_nT(Ts, alpha, t):
    undefined_t_vals = [0, Ts / (4 * alpha)]
    try:
    # if t in undefined_t_vals:
        # return lhopital(Ts, alpha, t)
    # else:
        return (1/sqrt(Ts)) * ((sin(pi*(1 - alpha) * t / Ts) + (4 * alpha * t / Ts) * cos(pi * (1 + alpha) * t / Ts))/(((pi*t)/Ts)*(1 - (4 * alpha * t / Ts)**2)))
    except ZeroDivisionError:
        return None
        # return lhopital(Ts, alpha, t)

def lhopital(Ts, alpha, t):
    numerator = (pi * (1 - alpha) / Ts) * cos(pi * (1 - alpha) * t / Ts) + (4 * alpha / Ts) * (cos(pi * (1 + alpha) * t / Ts) - (pi * (1 + alpha) * t / Ts) * sin(pi * (1 + alpha) * t / Ts))
    denominator = pi / sqrt(Ts) - (32 * pi * t / (Ts * sqrt(Ts)))
    return numerator / denominator

def NRZ(Ts):
    return [Ts]

def MANCH(Ts, Lp=60):
    return [-Ts, Ts] + [0] * Lp
```

### rx.py

```
from math import log2, sqrt, cos, sin
import numpy as np

def recieved_to_IQ(received_sig, freq):
    I = [i * sqrt(2) *  cos(freq) for i in received_sig]
    Q = [i * sqrt(2) * -sin(freq) for i in received_sig]

    return I, Q

def matched_filter(signal, pulse):
    pulse_reversed = list(reversed(pulse))
    return np.convolve(signal, pulse)

def sample_signal(sig, sample_time, Lp=0):
    sampled = []
    for idx in range(Lp,len(sig)-Lp,sample_time):
    # for idx in range(2*Lp-1,len(sig),sample_time):
        sampled.append(sig[idx])
    
    return sampled

def normalize_amplitude(signal, max_constellation_val):
    max_sig = max(signal)
    return [i * max_constellation_val / max_sig for i in signal]

def slice(I, Q, LUT):
    """
    LUT should be a 2D array where the amplitudes of '00' are in index 0 of the list as a 2 element list
    Returns a list of bits
    """
    num_bits = log2(len(LUT))       # Determine the number of bits per symnol
    sliced = []
    for idx in range(len(I)):
        distances = []
        for look in range(len(LUT)):    # Get a list of all distances
            distances.append(sqrt((I[idx] - LUT[look][0])**2 + (Q[idx] - LUT[look][1])**2))
        dist = np.array(distances)
        min_index = np.where(dist == dist.min())[0][0] # Get the minimimum distance as an index
        sliced += [int(i) for i in bin(min_index)[2:].zfill(int(num_bits))] # convert integer to bits
    return sliced
```


### tx.py

```
import numpy as np
import matplotlib.pyplot as plt
from plots import plot_and_show
from math import sqrt, cos, sin

def serial_to_parallel(bits:list, num_bits_per_point:int=1) -> list:
    """bits are the bits to be split up into seperate chunks, num_bits_per_point is 2 for 4 QAM, 4 for 16QAM, etc..."""
    to_return = []
    if len(bits) % num_bits_per_point != 0:
        print(f"Error: length of bits ({len(bits)} doesn't match the given number of bits per point ({num_bits_per_point})")

    for idx in range(int(len(bits)/num_bits_per_point)):
        parallel = 0
        for n in range(num_bits_per_point):
            parallel = (parallel << 1) | bits[idx * num_bits_per_point + n]
        to_return.append(parallel)
    return to_return

def parallel_to_I_Q(parallel_bits, LUT):
    """
    parallel_bits is a list of lists containing the parallel value ([0, 1, 3, 2, 0, 3, 1, ...])
    LUT is a Lookup Up Table where each is a list with len == 2 for I and Q, and index 0 = [I_0, Q_0] and index 1 = [I_1, Q_1], etc...
    """
    I = []
    Q = []
    for num in range(len(parallel_bits)):
        I.append(LUT[parallel_bits[num]][0])
        Q.append(LUT[parallel_bits[num]][1])

    return I, Q

def upsample(pulses, num_up):
    sig = np.zeros((len(pulses)*num_up,1))
    sig[range(0,len(pulses)*num_up,num_up)] = np.array(pulses).reshape(len(pulses),1)
    return sig[:,0]

def IQ_to_sig(I, Q, pulse:list, freq):
    I_upsampled = upsample(I)
    Q_upsampled = upsample(Q)

    plot_and_show(I_upsampled, title="I upsampled")
    plot_and_show(Q_upsampled, title="Q_upsampled")

    I_shaped = np.convolve(I_upsampled, pulse)
    Q_shaped = np.convolve(Q_upsampled, pulse)

    plot_and_show(I_shaped, title="I shaped")
    plot_and_show(Q_shaped, title="Q_shaped")

    I_sig = [i * sqrt(2) *  cos(freq) for i in I_shaped]
    Q_sig = [q * sqrt(2) * -sin(freq) for q in Q_shaped]

    plot_and_show(I_sig, title="I signal")
    plot_and_show(Q_sig, title="Q signal")

    soft = [I_sig[i] + Q_sig[i] for i in range(len(I_sig))]

    plot_and_show(soft, title="S(t)")

    return soft
```
